# Set up

In [ ]:
import pandas as pd
import numpy as np
import tqdm as tqdm

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

import pyro.distributions as dist


# Read Data

In [2]:
Y_data_train = pd.read_csv('../Data/simulation_results_train.txt', header=None, delimiter=' ')
Y_data_train = Y_data_train.drop(columns=[Y_data_train.columns[-1]])

Y_data_test = pd.read_csv('../Data/simulation_results_test.txt', header=None, delimiter=' ')
Y_data_test = Y_data_test.drop(columns=[Y_data_test.columns[-1]])

In [6]:
Y_data_max = pd.read_csv('simulation_results_5_max.txt', header=None, delimiter=' ')
Y_data_max = Y_data_max.drop(columns=[Y_data_max.columns[-1]])

In [3]:
RealCase = pd.read_csv('RealCase.csv', header=None, delimiter=',')

In [4]:
df_new = Y_data_train.drop(Y_data_train.columns[-18:], axis=1)
Y_data_train = df_new.drop(df_new.columns[17], axis=1)

In [5]:
df_new = Y_data_test.drop(Y_data_test.columns[-18:], axis=1)
Y_data_test = df_new.drop(df_new.columns[17], axis=1)

In [7]:
df_new = Y_data_max.drop(Y_data_max.columns[-18:], axis=1)
Y_data_max = df_new.drop(df_new.columns[17], axis=1)

In [6]:
Y_data_train.shape

(4196, 33)

## Save

In [8]:
np.savetxt("RealCase_Y_test.csv", Y_data_test, delimiter=",", fmt="%.8f")

## Check

In [8]:
col_min = np.nanmin(Y_data_train.values, axis=0)
col_max = np.nanmax(Y_data_train.values, axis=0)


((RealCase.values >= col_min) & (RealCase.values <= col_max)).sum(axis=1)

array([31])

In [9]:
mask = (RealCase.values >= col_min) & (RealCase.values <= col_max).ravel()

In [ ]:
mask[0]

(33,)

In [11]:
RealCase.shape

(1, 33)

In [13]:
RealCase_labels = pd.read_csv('AHA17_rows_full_labels_cir_then_rad.csv', delimiter=',')

In [37]:
RealCase_labels.columns[~mask[0]]

Index(['Basal-Anterior_rad', 'Apical-Lateral_rad'], dtype='object')

In [16]:
Y_data_train.values[:, ~mask[0]]

array([[0.15963 , 0.355223],
       [0.172847, 0.148745],
       [0.181315, 0.146777],
       ...,
       [0.193016, 0.254484],
       [0.219989, 0.262203],
       [0.11731 , 0.134579]], shape=(4196, 2))

In [17]:
TC = Y_data_train.values[:, ~mask[0]]

In [22]:
RealCase.values[0][~mask[0]]

array([1.00741986, 0.65111975])

In [23]:
RealCase

,0,1,2,3,4,5,6,7,8,9,...,23,24,25,26,27,28,29,30,31,32
0,188,-0.146816,-0.168986,-0.157946,-0.199671,-0.235694,-0.132681,-0.164577,-0.168338,-0.11218,...,0.508654,0.376972,0.58589,0.460637,0.596281,0.420518,0.402383,0.674607,0.65112,0.431821
